In [2]:
import pandas as pd

In [ ]:
df = pd.read_csv('./mini_mnist.csv')
df.head()

In [ ]:
X = df.drop('label', axis=1)
y = df['label']

In [1]:
from sklearn.decomposition import PCA

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
pca = PCA(n_components=2)  
pca.fit(X)

In [ ]:
X_pca = pca.transform(X)
X_pca_df = pd.DataFrame(X_pca, columns = ['PC1', 'PC2'])

In [ ]:
sns.set_palette('seismic')
sns.scatterplot(data=X_pca_df, x='PC 0', y='PC 1', hue=y, palette='Set1')